#### *Instalando as bibliotecas que serão necessárias*

In [30]:
# Instala a biblioteca "pandas" que será usada para manipulação de dados.
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [31]:
# Instala a biblioteca "numpy" que será usada para operações numéricas.
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [32]:
# Instala a biblioteca "tqdm" que será usada para a exibição de barras de carregamento.
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [33]:
# Instala a biblioteca "scikit-learn" que será usada para escalonamento das features.
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


#### *Importando as bibliotecas que serão necessárias*

In [34]:
# Importa a biblioteca "pandas", que será usada para manipulação de dados.
import pandas as pd

# Importa a biblioteca "numpy", que será usada para operações numéricas.
import numpy as np

# Importa o módulo log da biblioteca "math" para realizar cálculos envolvendo logarítmos na base e.
from math import log

# Importa a classe Ticker da "biblioteca" Ticker para lidar com ativos financeiros. 
from Ticker import Ticker

# Importa a classe Tickers da "biblioteca" Tickers para lidar com um conjunto de ativos financeiros.
from Tickers import Tickers

# Importa a classe Model da "biblioteca" Model para lidar com redes neurais do tipo LSTM.
from Model import Model

# Importa o módulo datetime da biblioteca datetime para lidar com datas.
from datetime import datetime

# Importa a biblioteca "tqdm", que será usada para a exibição de barras de carregamento.
from tqdm import tqdm

# Importa o módulo MinMaxScaler da biblioteca "sklearn", que será utilizado para realizar a "desnormalização" 
# das variáveis target que foram usadas no modelo LSTM.
from sklearn.preprocessing import MinMaxScaler

#### *Obtendo a lista de tickers do S&P500*

In [35]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_symbols= sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_symbols = sp500_symbols.tolist()

# Exibe a lista de tickets criada acima.
#sp500_symbols

#### *Definindo alguns parâmetros que serão importantes*

In [36]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais que este código necessita. 
'''

# Cria um dicionário para armazenar os parâmetros principais do modelo, garantindo flexibilidade e fácil modificação desses valores.
setup = {
    
    # Lista dos símbolos das ações que serão utilizadas para a coleta de dados e previsões.
    "symbols": sp500_symbols,
    
    # Data inicial para a extração de dados (Deve obrigatoriamente ser um dia de negociação).
    "data_extraction_initial_date": datetime(2019,1,2).date(), #  Primeiro dia de negociações do S&P500 em 2019.
    
    # Data final cujos dados serão coletados (Deve obrigatoriamente ser um dia de negociação e 
    # deve também obrigatoriamente suceder um dia de negociação.).
    "data_extraction_final_date": datetime(2023,12,30).date(), #(Último dia de negociações do S&P500 em 2023). 
    
    # Período da estratégia a ser usada: define a frequência para a análise dos retornos (Ex.: 5 para análise semanal, 20 para mensal).
    "strategy_time_period": 20, # Observação: São levados em conta como dias na estratégia apenas aqueles onde ocorreram negociações.
    
    # Limite mínimo de probabilidade para que um ativo seja considerado em regime de baixa volatilidade e, portanto, 
    # elegível para negociação no período em questão. Somente ativos cuja probabilidade média de baixa volatilidade 
    # exceda esse valor serão considerados negociáveis.
    "regime_probability_threshold": 0.9,
    
    # Quantidade máxima de ativos que poderão compor o portfólio em cada período.
    "number_of_tickets_to_be_selected": 50,
    
    # Define os períodos de tempo (janelas móveis) para o cálculo das principais features técnicas a serem usadas no modelo.
    "features_time_period": {
        # Define o período (janela móvel) para o cálculo dos retornos.
        "returns_time_period": 1,
        # Define o período (janela móvel) para o cálculo da média móvel exponencial.
        "exponential_moving_average_time_period": 14,
        # Define o período (janela móvel) para o cálculo do Índice de Força Relativa (RSI).
        "relative_strength_index_time_period": 14,
        # Define o período (janela móvel) para o cálculo da Faixa Média Verdadeira (ATR).
        "average_true_range_time_period": 14,
        # Define o período (janela móvel) para o cálculo do momentum.
        "momemtum_time_period": 14,
        # Frequência para coleta dos dados do VIX. O valor "1d" indica que os dados do VIX serão registrados 
        # em intervalos diários de negociação.
        "vix_time_period": "1d",
        # Define o período para o cálculo das mudanças percentuais (Usado no vix e no MSGARCH).
        "pct_change_period": 1,
    },
    # Define o comprimento das sequências de tempo (janelas móveis) usadas no modelo LSTM.
    "lstm_time_sequences_length": 3,
    #
    "output_file_name": "resultados_portfolio_mensal_full_6.xlsx"
}

#### *Obtendo os dados dos tickers*

In [ ]:
# Inicializa o objeto `tickers` com os parâmetros principais definidos no dicionário `setup`, incluindo símbolos das ações,
# datas de início e fim para extração de dados, períodos de cálculo das features, frequência da estratégia, 
# limite de probabilidade de regime e número de ativos selecionados.

# Todos os tickers presentes no objeto `tickers` terão os dados padronizados, ou seja, estarão alinhados com os mesmos índices de data,
# garantindo consistência temporal para cálculos e análises posteriores.

tickers = Tickers(setup['symbols'], setup['data_extraction_initial_date'], setup['data_extraction_final_date'],
                  setup['features_time_period'], setup['strategy_time_period'],
                  setup['regime_probability_threshold'], setup['number_of_tickets_to_be_selected'])

Baixando e processando os tickers:   0%|          | 0/503 [00:00<?, ?it/s]

$BF.B: possibly delisted; No price data found  (1d 2019-01-02 -> 2023-12-30)


#### *Criando o modelo LSTM*

In [37]:
# Calcula o número de features (colunas de dados) a serem utilizadas como entrada no modelo, subtraindo 1 para excluir a coluna do target.
features_number = (tickers.symbols[0].data.shape[1] - 1)

In [38]:
# Cria uma instância do modelo LSTM, passando o número de features e o comprimento das sequências de tempo (janelas temporais) para 
# preparar o modelo.
model = Model(features_number,setup['lstm_time_sequences_length'])

In [39]:
# Constrói a arquitetura da rede LSTM, configurando as camadas e os parâmetros necessários para o treinamento do modelo.
model.create_LSTM_model()

#### *Obtendo e salvando as predições*

In [40]:
def get_ticker_period_results(ticker: Ticker, predicted: pd.Series, period_initial_date: datetime.date, period_final_date: datetime.date) -> pd.DataFrame: 
    '''
        Description:
            Calcula e armazena os resultados de retorno predito e real para um ativo específico (`ticker`) em um período determinado.
            A função aplica a transformação inversa aos valores escalados (real e predito), calcula os retornos logarítmicos
            e retorna os resultados em um DataFrame.
        Args:
            ticker (Ticker): Objeto do tipo Ticker que contém dados e informações relevantes do ativo.
            predicted (pd.Series): Série temporal contendo os valores preditos do preço do ativo.
            period_initial_date (datetime.date): Data inicial do período de análise.
            period_final_date (datetime.date): Data final do período de análise.
        Return:
            pd.DataFrame: DataFrame com os resultados de retorno real e predito para o período especificado, 
                          contendo o símbolo da ação, retorno predito e retorno real.
    '''
    
    # Redimensiona y_train para que ele seja bidimensional (Necessário para o MinMaxScaler).
    resized_y_train = ticker.__y_train__.values.reshape(-1, 1)  

    # Cria uma instancia do MinMaxScaler para o target do ticker em questão.
    scaler_target = MinMaxScaler()

    # Ajusta a instância criada acima ao conjunto de treino do target do ticker em questão.
    scaler_target.fit_transform(resized_y_train)

    # Volta os valores de "y_test_scaled_sequences" para a escala normal e associa a esses valores as suas datas originais, criando assim
    # uma série temporal.
    real_y = pd.Series(scaler_target.inverse_transform(ticker.__y_test_scaled_sequences__).flatten(),
                    index= ticker.__y_test__.index[len(ticker.__y_test__) - len(ticker.__y_test_scaled_sequences__):]
    )
    
    # Adiciona um nome a série temporal "real_y".
    real_y.name = f"Preço real de {ticker.symbol}" # Útil para eventuais gráficos.

    # Volta os valores estimados para a escala normal e associa a esses valores as suas datas originais, criando assim
    # uma série temporal.
    predicted_y = pd.Series(
        scaler_target.inverse_transform(predicted).flatten(),
        index=ticker.__y_test__.index[len(ticker.__y_test__) - len(ticker.__y_test_scaled_sequences__):]
    )

    # Calcula o retorno predito e o retorno real como retornos logarítmicos e os salva no DataFrame results, juntamente do symbol do ticker.
    results = {
        "Ação": ticker.symbol,
        "Retorno Predito": log(predicted_y[-1]/predicted_y[0]) * 100,
        "Retorno Real": log(real_y[-1]/real_y[0]) * 100
    }

    # Define os índices do DataFrame para o período analisado. Tais índices estarão no formato "data inicial - data final".
    index = [str(period_initial_date) + " - " + str(period_final_date)]
    
    return pd.DataFrame(results, index=index)

In [41]:
def process_period(tickers: Tickers, tickers_to_trade:list, period_number:int, strategy_time_period: int, indexes: list, model) -> pd.DataFrame:
    '''
        Description:
            Processa os resultados do modelo para um certo período de negociação e um certo conjunto de tickers selecionados. 
            Para cada ticker, prepara os dados, treina o modelo e obtém as previsões para o período especificado.
            Retorna um DataFrame com os resultados de retorno predito e real, ordenados pelo retorno predito.
        Args:
            tickers (Tickers): Objeto contendo informações e dados históricos de todos os ativos.
            tickers_to_trade (list): Lista de índices dos symbols presentes na variável tickers que serão negociados no período atual.
            period_number (int): Número do período atual que está sendo processado.
            strategy_time_period (int): Período de tempo da estratégia em dias (usado para definir a janela de negociação).
            indexes (list): Lista de datas que representam os dias de negociação.
            model: Instância do modelo LSTM que será treinado e usado para previsões.
        Return:
            pd.DataFrame: DataFrame contendo os resultados dos retornos preditos e reais para os tickers negociados no período,
                          ordenados pelo retorno predito em ordem decrescente.
    '''
    
    # Define a data inicial do período atual a partir da lista de índices (indexes), multiplicando o número do período pelo tempo da estratégia.
    period_initial_date = indexes[strategy_time_period*period_number].date()
    # Define a data final do período atual a partir da lista de índices (indexes).
    period_final_date = indexes[strategy_time_period*period_number - 1 + strategy_time_period].date()
    
    # Inicializa um DataFrame vazio para armazenar os resultados do período atual.
    period_results = pd.DataFrame()
    
    # Itera sobre cada ticker na lista de tickers a serem negociados para o período.
    for ticker_index in tickers_to_trade: 
        
        # Prepara os dados do ticker em questão para o modelo LSTM, gerando sequências normalizadas de treino e teste.
        X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences = \
        tickers.symbols[ticker_index].prepare_data_for_lstm(period_initial_date,period_final_date,setup['lstm_time_sequences_length'])
        
        # Treina o modelo com os dados de treino e obtém as previsões para os dados de teste, junto com o erro quadrático médio do modelo.
        predicted, RMSE = model.train_model_and_get_results(
            X_train_scaled_sequences,X_test_scaled_sequences,y_train_scaled_sequences,y_test_scaled_sequences)
        
        # Calcula e armazena os retornos preditos e reais para o ticker no período atual.
        period_results = pd.concat([period_results,get_ticker_period_results(
            tickers.symbols[ticker_index], predicted, period_initial_date, period_final_date)])
    
    # Ordena o DataFrame dos resultados pelo retorno predito em ordem decrescente.
    period_results = period_results.sort_values(by="Retorno Predito", ascending=False)
        
    return period_results

In [42]:
def process_periods_and_save_results(tickers: Tickers, tickers_to_trade: list, periods_number: int, 
                                     indexes: list, model: Model, strategy_time_period: int, output_file_name: str) -> None:
    """
        Description:     
            Processa os resultados do modelo para uma certa quantidade de períodos de negociação e um certo conjunto de tickers 
            selecionados para cada período. Após isso, salva tais resultados em um arquivo Excel.
        Args:
            tickers (Tickers): Objeto contendo informações e dados históricos de todos os ativos.
            tickers_to_trade (list): Lista de índices dos symbols presentes na variável tickers que serão negociados no período atual.
            periods_number (int): Número total de períodos a serem processados.
            indexes (list): Lista de datas que representam os dias de negociação.
            model: Instância do modelo LSTM que será treinado e usado para previsões.
            strategy_time_period (int): Período de tempo da estratégia em dias (usado para definir a janela de negociação).
            output_file_name (str): Nome do arquivo de saída onde os resultados serão salvos.
        Return:
            Essa função não retorna nada, mas salva os resultados obtidos para os períodos em questão em um arquivo .xlsx.
    """
    
    # Inicializa uma lista para armazenar os resultados de cada período de negociação.
    results = []

    # Itera sobre o intervalo de períodos de negociação, com uma barra de progresso para monitorar o progresso da iteração.
    for period in tqdm(range(1, periods_number), desc="Processando os períodos", unit="período"):
        print(f"\nProcessando o período {period}...\n")
        
        # Verifica se existem tickers disponíveis para negociação no período atual.
        if tickers_to_trade[period] != []:
            # Se houver tickers para negociar, processa o período utilizando a função 'process_period'.
            period_results = process_period(tickers, tickers_to_trade[period], period, strategy_time_period, indexes, model)
            
            # Adiciona os resultados do período processado à lista de resultados.
            results.append(period_results)
        else:
            # Se não houver tickers para negociar, adiciona um DataFrame vazio na lista de resultados.
            results.append(pd.DataFrame())

    # Cria um arquivo Excel para salvar os resultados de todos os períodos.
    with pd.ExcelWriter(output_file_name) as writer:
        # Itera sobre os resultados e salva cada período em uma aba do Excel.
        for period, period_results in enumerate(results):
            # Define o nome da aba baseado no número do período.
            sheet_name = f'Período {period+1}'  
    
            # Salva o DataFrame do período na aba correspondente.
            period_results.to_excel(writer, sheet_name=sheet_name)  

In [43]:
# Calcula o número total de períodos de negociação que serão analisados. Para tal, é dividido o número total de dados 
# pelo período de tempo da estratégia.
periods_number = (tickers.symbols[0].data.shape[0] // setup['strategy_time_period'])

# Extrai o índice de datas dos dados históricos do primeiro ticker (aqui podemos assumir que todos os tickers têm o mesmo índice de datas).
indexes = tickers.symbols[0].data.index

# Obtém a lista de tickers a serem negociados para o período atual.
tickers_to_trade = tickers.get_tickers_to_trade()

In [44]:
# Chama a função 'process_periods_and_save_results' para processar e salvar, para todos os períodos, 
# os resultados obtidos através do modelo criado.

for i in range(7,10):
    aux = setup['output_file_name'].split('.')
    output_file_name = aux[0][:-2] + "_" + str(i) + ".xlsx"
    print(output_file_name)
    process_periods_and_save_results(tickers, tickers_to_trade, periods_number, indexes, 
                                model, setup['strategy_time_period'],output_file_name)

resultados_portfolio_mensal_full_7.xlsx


Processando os períodos:   0%|          | 0/61 [00:00<?, ?período/s]


Processando o período 1...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━

Processando os períodos:   2%|▏         | 1/61 [01:49<1:49:55, 109.92s/período]


Processando o período 2...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━

...


Processando o período 16...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━

Processando os períodos:  26%|██▌       | 16/61 [33:53<1:25:27, 113.95s/período]


Processando o período 17...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━

2024-11-12 08:15:53.725083: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Matrix size-incompatible: In[0]: [32,8], In[1]: [9,256]
	 [[{{function_node sequential_1_lstm_1_while_body_506891_rewritten}}{{node sequential_1/lstm_1/while/lstm_cell_1/MatMul}}]]
Processando os períodos:  26%|██▌       | 16/61 [36:29<1:42:37, 136.84s/período]


InvalidArgumentError: Graph execution error:

Detected at node sequential_1/lstm_1/while/lstm_cell_1/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/user/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/user/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/user/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/user/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/user/.local/lib/python3.10/site-packages/ipykernel/kernelb